In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Configurações Iniciais
BASE_URL = "https://dados.cvm.gov.br/dados/"
# Exemplo: Escolha as pastas que deseja mapear do site
FOLDERS_TO_DOWNLOAD = ['FI/DOC/CDA/', 'CIA_ABERTA/DOC/DFP/'] 
DEST_DIR = "dados_cvm"

def get_links(url, extension=".zip"):
    """Varre a página e retorna links que terminam com a extensão desejada."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Filtra links que não sejam de navegação de diretório (../)
        links = [urljoin(url, node.get('href')) for node in soup.find_all('a') 
                 if node.get('href').endswith(extension)]
        return links
    except Exception as e:
        print(f"Erro ao acessar {url}: {e}")
        return []

def download_file(url, folder_path):
    """Faz o download do arquivo mantendo o nome original."""
    os.makedirs(folder_path, exist_ok=True)
    file_name = url.split('/')[-1]
    path = os.path.join(folder_path, file_name)
    
    if not os.path.exists(path):
        print(f"Baixando: {file_name}...")
        r = requests.get(url)
        with open(path, 'wb') as f:
            f.write(r.content)
    else:
        print(f"Arquivo {file_name} já existe. Pulando.")

# --- Execução no Notebook ---
mapeamento = {folder: f"{BASE_URL}{folder}DADOS/" for folder in FOLDERS_TO_DOWNLOAD}

for pasta_nome, url_dados in mapeamento.items():
    print(f"\nProcessando: {pasta_nome}")
    
    # Busca por arquivos .zip ou .csv na subpasta DADOS
    arquivos = get_links(url_dados, ".zip") + get_links(url_dados, ".csv")
    
    for link in arquivos:
        # Salva seguindo a estrutura original de pastas
        save_path = os.path.join(DEST_DIR, pasta_nome, "DADOS")
        download_file(link, save_path)

print("\nConcluído!")